In [17]:
#!pip install transformers
#!pip install einops
#!pip install accelerate
#!pip install xformers

In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

#model = "tiiuae/falcon-40b"
model = "tiiuae/falcon-40b"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [20]:
prompt= """
Generate a SPARQL query that can be used to retrieve information from DBpedia. \n
For the input is a natural question <Question> that asks about some entity, property, or relation in DBpedia. \n
Here are some examples that shows you how to generate sparql correctly. 

<Examples>
"""
question= "Who is the president of Egypt?"

examples= "(Question: Which famous horserace has seen both Fergie sutherland and the illustrious War of Attrition? , SPARQL = SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/War_Of_Attrition_(horse)> <http://dbpedia.org/property/race> ?uri. <http://dbpedia.org/resource/Fergie_Sutherland> <http://dbpedia.org/property/race> ?uri . })\""
context= prompt+"\n"+examples+"\n(Question: "+question

In [21]:
sequences = pipeline(
   context,
    max_length=500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

/upb/users/m/manzoor/profiles/unix/cs/.local/lib/python3.11/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: 
Generate a SPARQL query that can be used to retrieve information from DBpedia. 

For the input is a natural question <Question> that asks about some entity, property, or relation in DBpedia. 

Here are some examples that shows you how to generate sparql correctly. 

<Examples>

(Question: Which famous horserace has seen both Fergie sutherland and the illustrious War of Attrition? , SPARQL = SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/War_Of_Attrition_(horse)> <http://dbpedia.org/property/race> ?uri. <http://dbpedia.org/resource/Fergie_Sutherland> <http://dbpedia.org/property/race> ?uri . })"
(Question: Who is the president of Egypt?, SPARQL = SELECT?uri?name WHERE { <http://dbpedia.org/resource/Egypt> <http://dbpedia.org/property/head_of_state>?uri.?uri <http://dbpedia.org/property/name>?name})

In the above example, you have the question, SPARQL query and the results.

Now, lets try some other questions that you can use to retrieve information from DBpedia.

<Qu

In [18]:
for seq in sequences: # 7B 500 Token after proper prompt
    print(seq)

{'generated_text': '\nGenerate a SPARQL query that can be used to retrieve information from DBpedia. \n\nFor the input is a natural question <Question> that asks about some entity, property, or relation in DBpedia. \n\nHere are some examples that shows you how to generate sparql correctly. \n\n<Examples>\n\n(Question: Which famous horserace has seen both Fergie sutherland and the illustrious War of Attrition? , SPARQL = SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/War_Of_Attrition_(horse)> <http://dbpedia.org/property/race> ?uri. <http://dbpedia.org/resource/Fergie_Sutherland> <http://dbpedia.org/property/race> ?uri . })"\n(Question: Who is the president of Egypt?, SPARQL = SELECT DISTINCT?uri WHERE { <http://rdfx.org/rdfx-2011/?p=http://rdfx.org/wiki/2011%20Presidential%20Election%20of%20Egypt&f=&c=&s=&d=&m=&y=&z=&x=&o=&u=&i=&t=&n=&q=&k=&v=&w=&u=&u=&w=&w=&t=0.1&v=0.1&w=&w=&w=&w=&o=&p=&w=&w=&v=&c=&u=&w=&u=&w=&v=&t=&v=&w=&v=&w=&w=&v=&t=&u=&u=&v=&v=&w=&w=&o=&o=&s=&s=&v=&v=&t

In [5]:
for seq in sequences: # 40B 200 Token
    print(seq)

{'generated_text': 'Write a SPARQL code for dbpedia that answer the following question: Who is the president of Egypt? Who is the president of USA? How old was the current president of USA on July 4th 2012? Who is the wife of president of USA?\n1 Answers\n1- SELECT?president WHERE (?country <>?country <>) AND?country <>; 2- SELECT?president WHERE (?country <>?country <>) AND (?country <> <>) AND (<> <> "Barack Obama"); 3- SELECT?president WHERE (?country <>?country <>) AND (?country <> <>) AND (<> <> "Barack Obama") AND (STR(?president) = "Michelle Obama"); 4- SELECT?president WHERE (?country <> '}


In [12]:
for seq in sequences: # 7B 500 token without proper prompt
    print(seq)

{'generated_text': 'Write a SPARQL code for dbpedia that answer the following question: Who is the president of Egypt? (The answer could be the name of any person, the name can be found on Wikipedia)\nFor those who need to use the data, I will provide a.rdf file with the answer of the question.\nPlease, I would like to have a solution that can be used with any language.\nThanks.\nI have found a solution that I have to share.\nThe question is:\nHow to use the dbpedia data in SPARQL?\nThe answer is:\nSELECT?p WHERE {?p <http:// dbpedia.org/resource/Mubarak>..\nYou can get the rdf file (the.rdf file has to be located inside the folder dbpedia) with this code:\n$rdf = readfile(\'/var/lib/dbpedia/Mubarak.rdf\');\nThen you can use this code:\n$query = "Select?p where {}"\n$result = dbpedia->getQuery($query, $dbpedia, $query->getParams());\n$p = $result->getResult();\nThis is a solution that you can use to get the answer of the question (in the rdf file).\nIf you need the code that I used for